In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import SGDClassifier,LogisticRegression
import datetime
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.metrics import classification_report
# Import feature selection and dimensionality reduction methods
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import TruncatedSVD
# Import metric for evaluation
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv('/kaggle/input/System-Threat-Forecaster/train.csv')

In [ ]:
data=pd.read_csv('/kaggle/input/System-Threat-Forecaster/train.csv')

In [ ]:
df.columns

In [ ]:
 # code for dropping the MachineID column as it is of no use for prediction
def drop_MachineID(df):
    df=df.drop(['MachineID'],axis=1)
    return df
df=drop_MachineID(df)

In [ ]:
numerical_cols = df.select_dtypes(include=['int64','float64'])

In [ ]:
numerical_cols

In [ ]:
plt.figure(figsize = (40,40))
sns.heatmap(numerical_cols.corr(), annot =  True, cmap = 'coolwarm')
plt.show()

below code checks with correlation that which are the most corrlated columns and out from that which one's are the one with lear corr with target that one we have to drop

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'numerical_cols' is your DataFrame with numerical columns and 'target' is your target variable
corr_matrix = numerical_cols.corr()

# Find pairs of features with correlation greater than or equal to 0.8
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):  # start from i+1 to avoid duplicate pairs
        if abs(corr_matrix.iloc[i, j]) >= 0.8:  # Using abs() to catch both positive and negative correlations
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

# Print the highly correlated pairs
print("Pairs of columns with correlation >= 0.8:")
for pair in high_corr_pairs:
    print(f"{pair[0]} and {pair[1]}: {pair[2]:.2f}")

# Create a list of all columns involved in high correlation pairs
to_drop = list(set([col for pair in high_corr_pairs for col in pair[:2]]))  # Using set() to remove duplicates

print("\nAll columns involved in high correlations:", to_drop)

# Investigate each pair and determine which column to drop
final_redundant_cols = []
for pair in high_corr_pairs:
    col1, col2 = pair[0], pair[1]
    if col1 in numerical_cols.columns and col2 in numerical_cols.columns:
        print(f"\nInvestigating pair: {col1} and {col2}")
        corr_with_target = numerical_cols[[col1, col2]].corrwith(numerical_cols['target'])
        print("Correlation with target:")
        print(corr_with_target)
        print("\nMissing values:")
        print(numerical_cols[[col1, col2]].isnull().sum())

        # Determine which column to drop (the one with lower correlation with target)
        if abs(corr_with_target[col1]) < abs(corr_with_target[col2]):
            final_redundant_cols.append(col1)
        else:
            final_redundant_cols.append(col2)
    else:
        print(f"\nOne or both columns '{col1}' and '{col2}' not found in DataFrame.")

# Remove duplicates from final_redundant_cols
final_redundant_cols = list(set(final_redundant_cols))

print("\nFinal list of redundant columns (lower correlation with target in each pair):")
print(final_redundant_cols)


In [ ]:
def drop_redundant(df):
    df=df.drop(['ProcessorManufacturerID', 'RealTimeProtectionState', 'OSBuildNumber', 'PrimaryDisplayResolutionVertical', 'OSUILocaleID'],axis=1)
    return df
df=drop_redundant(df)

dropping columns with no variation or variablity


In [ ]:
def drop_variablity(df):
    columns_with_one_unique_value = df.loc[:, df.nunique() == 1]
    df=df.drop(columns_with_one_unique_value,axis=1)
    return df

df=drop_variablity(df)

In [ ]:
df.describe

In [ ]:
#plts for checking outliers
sns.boxplot(df['TotalPhysicalRAMMB'])

In [ ]:
sns.boxplot(df['SystemVolumeCapacityMB'])

In [ ]:
sns.boxplot(df['PrimaryDisplayDiagonalInches'])

In [ ]:
sns.boxplot(df['PrimaryDiskCapacityMB'])

In [ ]:
import numpy as np

import numpy as np

def cap_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap the outliers
    df[column] = np.clip(df[column], lower_bound, upper_bound)
    
    return df

def handle_outliers(df):
    columns_to_clean = ['TotalPhysicalRAMMB', 'PrimaryDisplayDiagonalInches', 'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB']
    
    for column in columns_to_clean:
        if column in df.columns:
            # Handle NaN values (you can choose to fill them or leave as is)
            df[column] = df[column].fillna(df[column].median())  # Uncomment this line if you want to fill NaNs
            df = cap_outliers(df, column)
    
    return df

# Apply the function to your dataframe
df = handle_outliers(df)


In [ ]:
df.shape

extract the date components

In [ ]:
import pandas as pd

def extract_date(df, date_as_col, date_os_col):
    # Ensure the date columns are in datetime format
    df[date_as_col] = pd.to_datetime(df[date_as_col], errors='coerce')
    df[date_os_col] = pd.to_datetime(df[date_os_col], errors='coerce')

    # Handle NaT values gracefully
    df['DateAS_Days'] = df[date_as_col].dt.day.fillna(0).astype('Int64')  # Use Int64 to allow nulls
    df['DateAS_Year'] = df[date_as_col].dt.year.fillna(0).astype('Int64')
    df['DateAS_Month'] = df[date_as_col].dt.month.fillna(0).astype('Int64')

    df['DateOS_Days'] = df[date_os_col].dt.day.fillna(0).astype('Int64')
    df['DateOS_Year'] = df[date_os_col].dt.year.fillna(0).astype('Int64')
    df['DateOS_Month'] = df[date_os_col].dt.month.fillna(0).astype('Int64')

    # Optionally drop the original date columns
    df = df.drop([date_as_col, date_os_col], axis=1)

    return df

# Example usage
# Assuming 'df' is your DataFrame with 'DateAS' and 'DateOS' columns
df = extract_date(df, 'DateAS', 'DateOS')


In [ ]:
 df['DateAS_Month']

In [ ]:
df['DateOS_Days']

now firstly we will split the attribute to numerical and categorical and the we will extract different fields from the numerical part

In [ ]:
def split_attribute(attr):
    import re
    # Extract numerical part (numbers and dots at the beginning of the string)
    numerical_part = re.search(r'^[\d.]+', attr)
    numerical_part = numerical_part.group(0) if numerical_part else None
    
    # Extract categorical part (everything after the numerical part)
    categorical_part = re.sub(r'^[\d.]+', '', attr)
    return numerical_part, categorical_part

# Apply the function to split into two columns
df[['NumericalPart_OSBuildLab', 'CategoricalPart_OSBuildLab']] = df['OSBuildLab'].apply(lambda x: pd.Series(split_attribute(x)))
df=df.drop(['OSBuildLab','CategoricalPart_OSBuildLab'],axis=1)

In [ ]:
data[['NumericalPart_OSBuildLab', 'CategoricalPart_OSBuildLab']] = data['OSBuildLab'].apply(lambda x: pd.Series(split_attribute(x)))
data=data.drop(['OSBuildLab','CategoricalPart_OSBuildLab'],axis=1)

In [ ]:
def split_numeric_version_column(df, column_name):

    if column_name not in df.columns:
        print(f"Column '{column_name}' not found in DataFrame.")
        return df

    try:
        split_df = df[column_name].str.split('.', expand=True)

        # Convert all resulting columns to integers. Fill NaN if conversion fails.
        for col in split_df.columns:
          split_df[col] = pd.to_numeric(split_df[col], errors='coerce').astype('Int64')

        num_cols = split_df.shape[1]
        new_cols = [f'{column_name}_part_{i+1}' for i in range(num_cols)]
        df[new_cols] = split_df
        df = df.drop(column_name,axis=1)
        return df

    except AttributeError as e:
        print(f"Error splitting column '{column_name}': {e}. Column is not of type string")
        return df
    except ValueError as e:
        print(f"Error converting to numeric: {e}")
        return df
    except TypeError as e:
        print(f"Error splitting column '{column_name}': {e}")
        return df

df=split_numeric_version_column(df,'NumericalPart_OSBuildLab')


In [ ]:
data=split_numeric_version_column(data,'NumericalPart_OSBuildLab')

In [ ]:
data['NumericalPart_OSBuildLab_part_3']

now we will extract the categorical version from the string

In [ ]:
def split_version_column(df, column_name):
    """Splits a version string column into multiple numeric columns.

    Handles mixed data types and non-string values gracefully.
    """
    if column_name not in df.columns:
        print(f"Column '{column_name}' not found in DataFrame.")
        return df

    try:
        # Convert the column to string type first
        df[column_name] = df[column_name].astype(str)
        
        split_df = df[column_name].str.split('.', expand=True)

        # Convert all resulting columns to numeric. Use Int64 to handle potential NaNs.
        for col in split_df.columns:
            split_df[col] = pd.to_numeric(split_df[col], errors='coerce').astype('Int64')

        num_cols = split_df.shape[1]
        new_cols = [f'{column_name}_part_{i+1}' for i in range(num_cols)]
        df[new_cols] = split_df
        df = df.drop(columns=column_name)
        return df

    except AttributeError as e:
        print(f"Error splitting column '{column_name}': {e}. Column probably contains only NaN values")
        return df
    except ValueError as e:
        print(f"Error during conversion to numeric in column '{column_name}': {e}")
        return df
    except TypeError as e:
        print(f"Type error: {e}")
        return df
    
def split_all(df):
    df=split_version_column(df,'EngineVersion')
    df=split_version_column(df,'AppVersion')
    df=split_version_column(df,'SignatureVersion')
    df=split_version_column(df,'OSVersion')
    return df


df=split_all(df)

In [ ]:
df.columns


In [ ]:
df['SignatureVersion_part_4']

In [ ]:
y=df.target
df=df.drop(['target'],axis=1)

In [ ]:
numerical_cols = df.select_dtypes(include=['int64','float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

CLASSIFICATION using tranformers, pipelines, and different models

In [ ]:
numerical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Impute missing numerical values
        ('scaler', StandardScaler())                 # Scale numerical features
    ])

    # Categorical pipeline
categorical_pipeline = Pipeline([
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('hot',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
    ])

    # Full preprocessing pipeline using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
        ('numerical', numerical_pipeline, numerical_cols),
        ('categorical', categorical_pipeline, categorical_cols)
    ], remainder='passthrough') #remainder passthrough keeps the columns that are not specified in the transformers

In [ ]:
transformed_data = preprocessor.fit_transform(df)

In [ ]:
print(df.columns.tolist())


In [ ]:
# Create a DataFrame
df = pd.DataFrame(transformed_data,columns=preprocessor.get_feature_names_out())

In [ ]:
df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

In [ ]:
# model=LogisticRegression()
# model.fit(X_train,y_train)
# y_pred=model.predict(X_train)
# print(classification_report(y_train ,y_pred))

In [ ]:
# y_pred=model.predict(X_test)
# print(classification_report(y_test ,y_pred))

In [ ]:
# sgd_lasso = SGDClassifier(loss='hinge', penalty='l1', random_state=42, max_iter=1000, tol=1e-3) #loss='hinge' for linear SVM

# # Train the classifier
# sgd_lasso.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = sgd_lasso.predict(X_test)
# print(classification_report(y_test, y_pred))

In [ ]:
# model = DecisionTreeClassifier(random_state=42, max_depth=8) #max_depth to prevent overfitting
# # Train the classifier
# model.fit(X_train, y_train)
# y_pred=model.predict(X_train)
# print(classification_report(y_train, y_pred))

In [ ]:
# y_pred = model.predict(X_test)

# # Evaluate the classifier
# print(classification_report(y_test, y_pred))

# from sklearn.ensemble import RandomForestClassifier
# model=RandomForestClassifier(max_depth=8,n_estimators=20)
# model.fit(X_train,y_train)
# y_pred=model.predict(X_test)
# print(classification_report(y_test, y_pred))

In [ ]:
# from sklearn.linear_model import Perceptron
# perceptron_clf = Perceptron(
#     penalty=None,          # No regularization (L1 or L2)
#     alpha=0.1,          # Regularization strength (if penalty is set)
#     max_iter=10000,         # Maximum number of iterations (epochs)
#     tol=1e-3,              # Stopping criterion tolerance
#     random_state=42,
#     l1_ratio=1
# )

# # Step 4: Train the Perceptron model
# for i in range(10):
#     perceptron_clf.fit(X_train, y_train)

# # Step 5: Make predictions
# y_pred = perceptron_clf.predict(X_test)

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

In [ ]:
def feature_engineering(x_train, x_test, y_train, y_test):
    
    # A base model is defined
    base_model = LogisticRegression(max_iter=300)
    
    # Define feature engineering pipelines
    methods = {
        "Base_model": Pipeline([
            ('model', base_model)
        ], verbose = True),
        
        "Select_K_Best": Pipeline([
            ('feature_engineering', SelectKBest(score_func=f_classif, k=70)),
            ('model', base_model)
        ], verbose = True),
        
        "Truncated_SVD": Pipeline([
            ('feature_engineering', TruncatedSVD(n_components=70)),
            ('model', base_model)
        ], verbose = True)
        
     }

    # A variable to store the scores is set
    fe_scores = {}

    for name, method in methods.items():             # Iterate through each model in the 'methods' dictionary
        
        print(f"\n-----Processing with {name}-----")
        
        method.fit(x_train, y_train)                 # Fitting with various methods
        preds = method.predict(x_test)               # Predicting with sklearn's .predict method
        
        accuracy = accuracy_score(y_test, preds)     # Finding accuracy of prediction
        fe_scores[name] = accuracy                   # Storing accuracy in 'fe_scores'
        
        print(f"{name} Accuracy: {accuracy:.4f}\n")
        print(f"-----Process done : {name} -----\n")
    
    return fe_scores 

In [ ]:
feature_engineering(X_train,X_test,y_train,y_test)

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier  # For Gradient Boosting Classifier

# def model_training():
    
#     # Create a dictionary of models to be trained
#     models = {
#         'Logistic_Regression': LogisticRegression(max_iter=300),         # Logistic Regression model
#         'Gradient_Boosting_Classifier': GradientBoostingClassifier()     # Gradient Boosting Classifier model
#     }

#     m_scores = {}       # Dictionary to store accuracy scores of each model
#     predictions = {}    # Dictionary to store predictions of each model

#     # Iterate over each model in the models dictionary
#     for name, model in models.items():
#         print(f"-----------------------------")
#         print(f"Training with {name} model...")
        
#         # Create a pipeline with preprocessing and the current model
#         model_pipeline = Pipeline([
#             ('model', model)                   # Apply the model
#         ])
        
#         # Fit the pipeline on training data
#         model_pipeline.fit(X_train, y_train)
        
#         # Predict on test data
#         y_pred = model_pipeline.predict(X_test)
        
#         # Calculate accuracy score of the predictions
#         accuracy = accuracy_score(y_test, y_pred)
        
#         # Store the accuracy score and predictions in the dictionaries
#         m_scores[name] = accuracy
#         predictions[name] = y_pred
        
#         print(f"{name} Accuracy: {accuracy:.4f}\n")
#         print(f"Model training done for: {name}")
      
#     # Return the accuracy scores and predictions
#     return m_scores, predictions

In [ ]:
# pca=PCA(n_components=80)
# X_train_=pca.fit_transform(X_train)
# X_test_=pca.transform(X_test)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# threshold = .98
# optimal_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= threshold) + 1
# print(f"Number of components to capture {threshold*100}% variance: {optimal_components}")

In [ ]:
# m_scores,predictions=model_training()

In [ ]:
# Y_pred=predictions['Gradient_Boosting_Classifier']
# print(classification_report(Y_pred,y_test))

In [ ]:
# model=GradientBoostingClassifier()
# model.fit(X_train,y_train)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
model = XGBClassifier(
    colsample_bytree=0.9,
    gamma=0,
    reg_alpha=0,
    reg_lambda=2,
    learning_rate=.1,
    max_depth=4,            # Note: max_depth should be an integer (not 0.3)
    n_estimators=550,        # Correct parameter name is n_estimators
    subsample=0.8,
    use_label_encoder=False,  # Suppress label encoder warnings
    eval_metric='logloss'     # Set evaluation metric to avoid warnings
)

In [ ]:
# model = XGBClassifier()
# param_grid = {
#     'n_estimators': [500, 800],  # Reduce range
#     'max_depth': [2,3],        # Smaller range
#     'learning_rate': [0.1],
#     'subsample': [0.8],         # Fix to one value
#     'colsample_bytree': [0.8],
#     'gamma': [0],          # Limit values
#    'reg_alpha': [0, 0.01, 0.1],          # L1 regularization
#     'reg_lambda': [1, 2, 5] 
# }

# #Initialize GridSearchCV
# grid_search = GridSearchCV(
#     estimator=model,
#     param_grid=param_grid,
#     cv=3,                                # 3-fold cross-validation
#     n_jobs=-1,                           # Use all available cores
#     verbose=2                            # Progress updates
# )

# if X_train is not None and y_train is not None:
#     print("Starting GridSearchCV for XGBClassifier...")

#     # Fit GridSearchCV
#     grid_search.fit(X_train, y_train)

In [ ]:
# Display the best parameters
# print("Best Parameters:", grid_search.best_params_)

# # Display the best cross-validation score
# print("Best Cross-Validation Score:", grid_search.best_score_)

# # Display the best estimator (the model with the best parameters)
# print("Best Estimator:", grid_search.best_estimator_)

# # Access the detailed cross-validation results
# cv_results = grid_search.cv_results_

# # Show the keys in cv_results_ for reference
# print("Keys in cv_results_:", cv_results.keys())

# # Example: Display mean test scores for each parameter combination
# mean_test_scores = cv_results['mean_test_score']
# print("Mean Test Scores for Each Parameter Combination:", mean_test_scores)

# # Combine parameter combinations and scores in a DataFrame (optional)
# results_df = pd.DataFrame(cv_results)
# print(results_df[['params', 'mean_test_score', 'std_test_score']])


In [ ]:
# model=grid_search.best_estimator_
model.fit(X_train,y_train)
y_pred=model.predict(X_train)
print(classification_report(y_pred,y_train))

In [ ]:
y_pred=model.predict(X_test)
print(classification_report(y_pred,y_test))

PERFORM ALL THE SAME OPERATIONS ON THE TEST DATA AS PERFORMED ON THE TTRAIN DATA

In [ ]:
test_data=pd.read_csv('/kaggle/input/System-Threat-Forecaster/test.csv')
test_data.describe

In [ ]:

test_data=drop_MachineID(test_data)
test_data=drop_redundant(test_data)
test_data=drop_variablity(test_data)

test_data=handle_outliers(test_data)
test_data=extract_date(test_data,'DateAS','DateOS')
test_data[['NumericalPart_OSBuildLab', 'CategoricalPart_OSBuildLab']] = test_data['OSBuildLab'].apply(lambda x: pd.Series(split_attribute(x)))
test_data=test_data.drop(['OSBuildLab','CategoricalPart_OSBuildLab'],axis=1)
test_data=split_numeric_version_column(test_data,'NumericalPart_OSBuildLab')
test_data=split_all(test_data)
transformed_data = preprocessor.transform(test_data)

In [ ]:
test_data = pd.DataFrame(transformed_data,columns=preprocessor.get_feature_names_out())

In [ ]:
y_pred=model.predict(test_data)
submission=pd.DataFrame({'id':range(0,test_data.shape[0]),
                        'target':y_pred})

submission.to_csv('submission.csv',index=False)
print("hi")

In [ ]:
len(y_pred)